In [29]:
# Import the necessary libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Lambda, LSTM, RepeatVector, TimeDistributed, Flatten, Reshape
from keras.models import Model
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.losses import binary_crossentropy
from keras import backend as K
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from custom_penalty import custom_penalty

# Load the original dataset
processed_data = pd.read_csv('Processed Data/Aruba_17/processed_data.csv')
# Find the maximum number that can be evenly divisible by 32, given the length of the dataset
max_length = len(processed_data) - len(processed_data) % 32
# processed_data = processed_data.head(16000)
# only use all the rows up to the max_length
processed_data = processed_data.head(3200)
# Extract the relevant columns from the dataset
timestamp = processed_data['Timestamp'].values
device_id = processed_data['Device ID'].values
status = processed_data['Status'].values
activity = processed_data['Activity'].values
activity_status = processed_data['Activity Status'].values

In [ ]:
# X = np.stack((timestamp, device_id, status, activity, activity_status), axis=1)

# # Normalize the data using z-score normalization
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

# # Scale the values to be within the range of 0 to 1
# min_max_scaler = MinMaxScaler()
# X = min_max_scaler.fit_transform(X)

In [31]:
# Define the log directory for TensorBoard
# log_dir = "logs/"

# Create a callback for TensorBoard
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Prepare the data for input into the VAE model
X = np.stack((timestamp, device_id, status, activity, activity_status), axis=1)

# Normalize the data using minMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
# print(X.head(20))
# Find the corresponding normalized values
temp_min_max_values = np.zeros((4, 5))
temp_min_max_values[0, 0] = 0
temp_min_max_values[1, 0] = processed_data['Timestamp'].nunique() - 1

# M sensors
temp_min_max_values[0, 1] = 3
temp_min_max_values[1, 1] = 33
temp_min_max_values[0, 2] = 54
temp_min_max_values[1, 2] = 55  # 'OFF' and 'ON' status values for M sensors

# T sensors
temp_min_max_values[2, 1] = 34
temp_min_max_values[3, 1] = 38
temp_min_max_values[2, 2] = 0
temp_min_max_values[3, 2] = 43  # Temperature values for T sensors

# D sensors
temp_min_max_values[0, 1] = 0
temp_min_max_values[1, 1] = 2
temp_min_max_values[0, 2] = 53
temp_min_max_values[1, 2] = 56  # 'CLOSE' and 'OPEN' status values for D sensors

temp_min_max_values[:, 4] = [0, 2, 0, 2]     # Activity Status values

# Find the corresponding normalized values using the scaler
normalized_min_max_values = scaler.transform(temp_min_max_values)

# Use KMeans to cluster sequences into 14 different groups
# kmeans = KMeans(n_clusters=14, random_state=0)
# clusters = kmeans.fit_predict(X)

# Split the data into training and testing sets
batch_size = 32
validation_split = 0.2
timesteps = 128 # number of previous records considered
input_dim = X.shape[1] # number of features, there are 5 features in the dataset

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, X, test_size=validation_split, shuffle=False)

# Pad the data to ensure it is divisible by the desired shape
remainder_train = X_train.shape[0] % (batch_size * timesteps)
if remainder_train > 0:
    X_train = np.concatenate([X_train[:-remainder_train], np.zeros((batch_size * timesteps - remainder_train, input_dim))])
    y_train = np.concatenate([y_train[:-remainder_train], np.zeros((batch_size * timesteps - remainder_train, input_dim))])

remainder_val = X_val.shape[0] % (batch_size * timesteps)
if remainder_val > 0:
    X_val = np.concatenate([X_val[:-remainder_val], np.zeros((batch_size * timesteps - remainder_val, input_dim))])
    y_val = np.concatenate([y_val[:-remainder_val], np.zeros((batch_size * timesteps - remainder_val, input_dim))])

# Reshape the datasets to have the correct shape for the model
X_train = X_train.reshape((-1, timesteps, input_dim))
y_train = y_train.reshape((-1, timesteps, input_dim))

X_val = X_val.reshape((-1, timesteps, input_dim))
y_val = y_val.reshape((-1, timesteps, input_dim))

latent_dim = 2
encoding_dim = 32

# ==================== ENCODER ====================
inputs = Input(batch_shape=(batch_size, timesteps, input_dim), name='encoder_input')
x = LSTM(encoding_dim*2, return_sequences=True)(inputs)
x = LSTM(encoding_dim, return_sequences=False)(x) 
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)
# z_mean is the mean of the latent space
# z_log_var is the variance of the latent space

def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
# encoder.summary()

# ================= LATENT SPACE ==================
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
# ==================== DECODER ====================
x = Dense(timesteps * encoding_dim, activation='relu')(latent_inputs)
x = Reshape((timesteps, encoding_dim))(x)
x = LSTM(encoding_dim, return_sequences=True, input_shape=(timesteps, encoding_dim))(x)
x = TimeDistributed(Dense(input_dim))(x)
# LSTM layer in the decoder is used to reconstruct the original sequence

# the VAE model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
decoder = Model(latent_inputs, x, name='decoder')
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae')

def custom_penalty(y_pred, normalized_min_max_values, input_dim):
    y_pred = K.reshape(y_pred, (-1, input_dim))

    y_pred_device_id = y_pred[:, 1]
    y_pred_status = y_pred[:, 2]

    penalties = []
    for i in range(3):
        min_device_id = K.reshape(normalized_min_max_values[i * 2, 1], K.shape(y_pred_device_id))
        max_device_id = K.reshape(normalized_min_max_values[i * 2 + 1, 1], K.shape(y_pred_device_id))
        min_status = K.reshape(normalized_min_max_values[i * 2, 2], K.shape(y_pred_status))
        max_status = K.reshape(normalized_min_max_values[i * 2 + 1, 2], K.shape(y_pred_status))

        device_penalty = K.sum(K.relu(y_pred_device_id - max_device_id) + K.relu(min_device_id - y_pred_device_id))
        status_penalty = K.sum(K.relu(y_pred_status - max_status) + K.relu(min_status - y_pred_status))

        penalties.append(device_penalty + status_penalty)

    penalty = K.mean(K.stack(penalties, axis=0))

    return penalty

# Loss function
reconstruction_loss = binary_crossentropy(K.flatten(inputs), K.flatten(outputs))
reconstruction_loss *= timesteps * input_dim
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

# Add the custom penalty to the loss function
penalty_weight = 10.0  # Adjust the weight of the penalty term as needed
# penalty = custom_penalty(y_pred=outputs)
penalty = custom_penalty(outputs, normalized_min_max_values, input_dim)
penalty *= penalty_weight

vae_loss = K.mean(reconstruction_loss + kl_loss + penalty)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')
# vae.summary()
# callbacks=[tensorboard_callback]
num_epochs = 300
history = vae.fit(X_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val, y_val))

# plot_model(vae, to_file='model.png', show_shapes=True)
# Use the encoder to generate embeddings for each sequence
encoder_model = Model(inputs, z_mean)
# print(encoder_model.layers[0].input_shape)

X_embedded = encoder_model.predict(X_train, batch_size=batch_size)

Epoch 1/300


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/vae/tf.keras.backend.binary_crossentropy_7/mul_1/Mul_1' defined at (most recent call last):
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2800.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2800.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Alec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Alec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start(

In [10]:
# Generate a fake dataset using the VAE model
n_samples = len(processed_data)

# Sample from the latent space
z_samples = np.random.normal(size=(n_samples, latent_dim))

# Use the decoder to generate the output
predicted_values = decoder.predict(z_samples)
predicted_values = np.reshape(predicted_values, (n_samples, timesteps, input_dim))

# Undo the normalization
predicted_values = np.reshape(predicted_values, (-1, input_dim))
predicted_values = scaler.inverse_transform(predicted_values)

# Round each of the values in the array to the nearest integer
predicted_values = np.rint(predicted_values)

# Save the prediction data to a new file 'predicted_Data.csv'
predicted_data = pd.DataFrame(predicted_values.reshape((-1, input_dim)), columns=['Timestamp', 'Device ID', 'Status', 'Activity', 'Activity Status'])

with open('Predictions/Aruba_17_prediction.txt', 'w') as file:
    for _, row in predicted_data.iterrows():
        file.write(','.join(map(str, row.values)) + '\n')

3200
100/100 [==============================] - 1s 10ms/step


In [ ]:
# Plot the training and validation loss with x and y labels, and a grid
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.legend()
# Validation loss > training loss, underfitting
# validation loss > training loss, overfitting, if it decreases and then increases again.
# If they both decreease and stabilize at a specific point, it is an optimal fit.

# Plot the evaluation loss vs the iterations
plt.plot(history.history['loss'], label='Training loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()


In [ ]:
# # Plot the model
# from keras.utils import plot_model

# # Display the layers, number of layers, number of nodes etc
# plot_model(vae, to_file='vae.png', show_shapes=True, show_layer_names=True)

# # Load the image and display it
# img = plt.imread('vae.png')
# plt.figure(figsize=(16, 12))
# plt.imshow(img)
# plt.axis('off')
# plt.show()
